Install all the depedencies.

In [39]:
pip install requests beautifulsoup4 nltk pandas openpyxl xlsxwriter

     ---------------------------------------- 0.0/153.0 kB ? eta -:--:--
     -------- ------------------------------- 30.7/153.0 kB ? eta -:--:--
     -------- ------------------------------- 30.7/153.0 kB ? eta -:--:--
     --------- --------------------------- 41.0/153.0 kB 326.8 kB/s eta 0:00:01
     --------- --------------------------- 41.0/153.0 kB 326.8 kB/s eta 0:00:01
     -------------- ---------------------- 61.4/153.0 kB 272.3 kB/s eta 0:00:01
     ------------------- ----------------- 81.9/153.0 kB 305.0 kB/s eta 0:00:01
     ------------------- ----------------- 81.9/153.0 kB 305.0 kB/s eta 0:00:01
     ---------------------- -------------- 92.2/153.0 kB 238.1 kB/s eta 0:00:01
     -------------------------- --------- 112.6/153.0 kB 261.9 kB/s eta 0:00:01
     -------------------------- --------- 112.6/153.0 kB 261.9 kB/s eta 0:00:01
     ---------------------------- ------- 122.9/153.0 kB 225.3 kB/s eta 0:00:01
     ---------------------------- ------- 122.9/153.0 kB 22


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\Shyamalan Kannan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Importing all the required libraries.

In [27]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Shyamalan
[nltk_data]     Kannan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Shyamalan
[nltk_data]     Kannan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import re
import os
import xlsxwriter
def retrieve_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('title').get_text() if soup.find('title') else ''
        paragraphs = soup.find_all('p')
        article_text = ' '.join(p.get_text() for p in paragraphs)

        return title, article_text

    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None, None

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def calculate_sentiment_percentage(text, positive_words, negative_words):
    words = text.lower().split()
    total_words = len(words)
    positive_count = sum(1 for word in words if word in positive_words)
    negative_count = sum(1 for word in words if word in negative_words)
    positive_percentage = (positive_count / total_words) * 100
    negative_percentage = (negative_count / total_words) * 100
    return positive_percentage, negative_percentage

def calculate_polarity_score(positive_percentage, negative_percentage):
    return positive_percentage - negative_percentage

def calculate_subjectivity_score(positive_percentage, negative_percentage):
    return positive_percentage + negative_percentage

def calculate_avg_sentence_length(text):
    sentences = sent_tokenize(text)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    total_sentences = len(sentences)
    return total_words / total_sentences if total_sentences > 0 else 0

def calculate_percentage_complex_words(text):
    words = word_tokenize(text)
    complex_word_count = sum(1 for word in words if len(word) > 6)  # Example threshold for complexity
    total_words = len(words)
    return (complex_word_count / total_words) * 100 if total_words > 0 else 0

def calculate_fog_index(avg_sentence_length, percentage_complex_words):
    return 0.4 * (avg_sentence_length + percentage_complex_words)

def calculate_avg_words_per_sentence(text):
    sentences = sent_tokenize(text)
    total_sentences = len(sentences)
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    return total_words / total_sentences if total_sentences > 0 else 0

def calculate_complex_word_count(text):
    words = word_tokenize(text)
    complex_word_count = sum(1 for word in words if len(word) > 6)  # Example threshold for complexity
    return complex_word_count

def calculate_word_count(text):
    words = word_tokenize(text)
    return len(words)

def count_syllables(word):
    vowels = "aeiouy"
    count = 0
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count = 1
    return count

def calculate_syllable_per_word(text):
    words = word_tokenize(text)
    total_syllables = sum(count_syllables(word) for word in words)
    total_words = len(words)
    return total_syllables / total_words if total_words > 0 else 0

def calculate_personal_pronouns(text):
    personal_pronouns = ['i', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours', 'you', 'your', 'yours', 'he', 'him', 'his', 'she', 'her', 'hers', 'it', 'its', 'they', 'them', 'their', 'theirs']
    words = word_tokenize(text)
    personal_pronoun_count = sum(1 for word in words if word.lower() in personal_pronouns)
    return personal_pronoun_count

def calculate_avg_word_length(text):
    words = word_tokenize(text)
    total_characters = sum(len(word) for word in words)
    total_words = len(words)
    return total_characters / total_words if total_words > 0 else 0

input_file_path = 'Input.xlsx'
output_data_file = 'Output Data Structure.xlsx'
output_folder = 'extracted_articles'

positive_words = load_word_list('positive-words.txt')
negative_words = load_word_list('negative-words.txt')

data_frame = pd.read_excel(input_file_path)
os.makedirs(output_folder, exist_ok=True)

output_data = []

for index, row in data_frame.iterrows():
    url = row['URL']
    url_id = re.sub(r'\W+', '', url)

    title, article_text = retrieve_text_from_url(url)

    if title and article_text:
        article_text = remove_stopwords(article_text)
        positive_percentage, negative_percentage = calculate_sentiment_percentage(article_text, positive_words, negative_words)
        polarity_score = calculate_polarity_score(positive_percentage, negative_percentage)
        subjectivity_score = calculate_subjectivity_score(positive_percentage, negative_percentage)
        avg_sentence_length = calculate_avg_sentence_length(article_text)
        percentage_complex_words = calculate_percentage_complex_words(article_text)
        fog_index = calculate_fog_index(avg_sentence_length, percentage_complex_words)
        avg_words_per_sentence = calculate_avg_words_per_sentence(article_text)
        complex_word_count = calculate_complex_word_count(article_text)
        word_count = calculate_word_count(article_text)
        syllable_per_word = calculate_syllable_per_word(article_text)
        personal_pronouns = calculate_personal_pronouns(article_text)
        avg_word_length = calculate_avg_word_length(article_text)

        output_data.append({
            'URL': url,
            'POSITIVE SCORE': positive_percentage,
            'NEGATIVE SCORE': negative_percentage,
            'POLARITY SCORE': polarity_score,
            'SUBJECTIVITY SCORE': subjectivity_score,
            'AVG SENTENCE LENGTH': avg_sentence_length,
            'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
            'FOG INDEX': fog_index,
            'AVG NUMBER OF WORDS PER SENTENCE': avg_words_per_sentence,
            'COMPLEX WORD COUNT' : complex_word_count,
            'WORD COUNT' : word_count,
            'SYLLABLE PER WORD' : syllable_per_word,
            'PERSONAL PRONOUNS' : personal_pronouns,
            'AVG WORD LENGTH' : avg_word_length
        })
        
workbook = xlsxwriter.Workbook(output_data_file)
worksheet = workbook.add_worksheet()
hyperlink_format = workbook.add_format({'color': 'blue', 'underline': 1})
worksheet.set_column('A:A', 15)
worksheet.write_url('A1', output_data_file, hyperlink_format)
workbook.close()

output_data_frame = pd.DataFrame(output_data)
output_data_frame.to_excel(output_data_file, index=False)
print(f"Output data saved to {output_data_file}")

Error: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Error: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
Output data saved to Output Data Structure.xlsx
